# Triggers

## Scheduled Trigger

Check created events in AmazonEventBridge->Events->Rules in your AWS console.

**Parameters for scheduled trigger**

In [11]:
rule_name = 'DailyKGPipelineTrigger'
pipeline_id = "Id26b29827-319a-45b4-8726-f33a37e5f22b"
pipeline_arn = "arn:aws:sagemaker:us-east-1:093729152554:pipeline/ckgqa-p-kiqtyrraeiec1631235879"
run_pipeline_role_arn = "arn:aws:iam::093729152554:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole"

Create a rule

In [8]:
import boto3

events = boto3.client('events')

# use the same name will update the trigger
events.put_rule(
    Name=rule_name,
    ScheduleExpression='rate(1 day)',
    State='DISABLED',
    Description='Daily re-run the knowledge graph generation pipeline',
    EventBusName='default'
)

{'RuleArn': 'arn:aws:events:us-east-1:093729152554:rule/DailyKGPipelineTrigger',
 'ResponseMetadata': {'RequestId': 'ee572c89-eee7-4e29-8dde-a0cad909eb8e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ee572c89-eee7-4e29-8dde-a0cad909eb8e',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '79',
   'date': 'Tue, 14 Sep 2021 05:55:07 GMT'},
  'RetryAttempts': 0}}

Add the code pipeline as a target:

In [12]:
events.put_targets(
    Rule=rule_name,
    EventBusName='default',
    Targets=[
        {
            "Id": pipeline_id,
            "Arn": pipeline_arn,
            "RoleArn": run_pipeline_role_arn
        }
    ]
)

{'FailedEntryCount': 0,
 'FailedEntries': [],
 'ResponseMetadata': {'RequestId': '818aed89-52aa-4743-b5d5-71fcdc958ef1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '818aed89-52aa-4743-b5d5-71fcdc958ef1',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '41',
   'date': 'Tue, 14 Sep 2021 06:06:56 GMT'},
  'RetryAttempts': 0}}

## S3 Trigger